#training model on the notreal

In [14]:
# from sb3_contrib import  RecurrentPPO
# unreal_env=Notreal()
# model=RecurrentPPO("MlpLstmPolicy",env= unreal_env
#                    ,learning_rate=2*3e-4 , batch_size=256 ,n_epochs=20)

In [15]:
# model.learn(total_timesteps=1000000,progress_bar=True)
# model.save("model_notreal")

#with added distrubences as stated in the paper, recheck the equations

In [16]:
# model.learn(total_timesteps=1000000,progress_bar=True)
# model.save("model_notreal2")
# model.learn(total_timesteps=1000000,progress_bar=True)
# model.save("model_notreal3")
# model.learn(total_timesteps=1000000,progress_bar=True)
# model.save("model_notreal4")
# model.learn(total_timesteps=1000000,progress_bar=True)
# model.save("model_notreal5")

#training SAC

In [17]:
enviorment = Notreal(render_mode="human")
print(enviorment.render_mode)

human


In [18]:
with open("log.txt", 'a') as f:
    f.write("Hello, World!\n")

In [6]:
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback

# class RenderCallback(BaseCallback):
#     """
#     A custom callback that renders the environment during training.
#     """
#     def __init__(self, render_freq=1000, verbose=0):
#         super(RenderCallback, self).__init__(verbose)
#         self.render_freq = render_freq

#     def _on_step(self) -> bool:
#         if self.n_calls % self.render_freq == 0:
#             self.training_env.render()
#         return True

class RenderCallback(BaseCallback):
    def __init__(self, render_freq=1000, verbose=0, log_file='log.txt'):
        super(RenderCallback, self).__init__(verbose)
        self.render_freq = render_freq
        self.log_file = log_file

    def _on_step(self) -> bool:
        if self.n_calls % self.render_freq == 0:
            render_output = self.training_env.render()
            with open(self.log_file, 'a') as f:
                f.write(f"Step: {self.n_calls}, Render Output: {render_output}\n")
        return True
# Initialize the environment

# Create the SAC model
model = SAC(
    "MlpPolicy", enviorment, 
    learning_rate=3e-4,     
    buffer_size=int(1e6), 
    batch_size=1024,       
    tau=0.005,            
    gamma=0.90,           
    train_freq=(100 , "step"),         
    gradient_steps=-1,     
    ent_coef='auto',
    verbose=1         # Enable logging
)

# Create a callback that renders the environment every 1000 steps
render_callback = RenderCallback(render_freq=1000)

# Train the model with the render callback
model_SAC.learn(total_timesteps=1000000,progress_bar=False, callback=render_callback)
model_SAC.save("SAC_Case 3 (REward changed).zip")
# model_SAC.learn(total_timesteps=500000,progress_bar=True)
# model_SAC.save("SAC_Case 3 V2.zip")
# model_SAC.learn(total_timesteps=500000,progress_bar=True)
# model_SAC.save("SAC_Case 3 V3.zip")



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Current state: [  3.65840889 824.70424258   0.        ] Total_Reward=-1000 


KeyboardInterrupt: 

In [26]:
model_SAC.save("SAC_Case 3 V3.zip")
model_SAC.learn(total_timesteps=10000,callback=render_callback)


Current state: [  3.65159255 532.85061054   0.        ] Total_Reward=-1000

Current state: [ 7.40328152 78.70300807  0.        ] Total_Reward=-2000

Current state: [1.03954732e+01 6.67758417e+01 1.31886933e-03] Total_Reward=-2873.0437553032725

Current state: [1.40076665e+01 5.64860169e+01 1.22869499e-02] Total_Reward=-2185.5206558228515

Current state: [1.78975947e+01 4.33991851e+01 2.23957321e-02] Total_Reward=-445.579060277165

Current state: [2.22273783e+01 3.76505040e+01 3.20360951e-02] Total_Reward=2271.3393856264074

Current state: [2.66928328e+01 4.73477027e+01 4.02826789e-02] Total_Reward=5891.030260334229

Current state: [31.64216781 39.8258287   0.04948863] Total_Reward=10386.981542755448

Current state: [37.24667525 40.81319028  0.05973658] Total_Reward=15857.008430633294

Current state: [43.03736197 40.61154404  0.06900284] Total_Reward=22285.987627422557

#how about scalling


In [43]:
rewards=[]
for t in range(1000000):
    obs = enviorment.reset()[0]
    action, _states = model_SAC.predict(obs)
    obs, reward, done, _, _ = enviorment.step(action)
    rewards.append(reward)

KeyboardInterrupt: 

In [45]:
sum(rewards)

-413885

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class THIRD_ENV_WITH_DISTURBENCE(gym.Env):
    def __init__(self):
        super(THIRD_ENV_WITH_DISTURBENCE, self).__init__()

        # State space: concentrations of biomass (C_X), nitrate (C_N), and product (C_q)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(3,), dtype=np.float32)
        
        # Action space: light intensity (I) and inflow rate (F_N)
        self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([400, 40]), dtype=np.float32)

        # Initial states
        self.state = None
        self.time_step = 0
        self.max_time_steps = 10  # Define the number of time steps in the batch

        # Disturbance and noise parameters
        self.sigma_d = np.array([4e-3, 1.0, 1e-7])  # 
        self.sigma_n = np.array([4e-4, 0.1, 1e-8])  

        
        self.penalty_coefficients = np.array([3.125e-8, 3.125e-6])

    def reset(self):
        # Initial concentrations (state)
        self.state = np.array([1.0, 150.0, 0.0], dtype=np.float32)
        self.time_step = 0
        return self.state + np.random.normal(0, self.sigma_n)  # Add initial measurement noise

    def step(self, action):
        # Apply system dynamics and additive disturbance
        I, F_N = action

        C_X, C_N, C_q = self.state

        dC_X = (I / (I + 178.9)) * C_X * (C_N / (C_N + 393.1)) - 0.0572 * C_X
        dC_N = -504.1 * (I / (I + 178.9)) * C_X * (C_N / (C_N + 393.1)) + F_N
        dC_q = (0.00016 * (I / (I + 23.51)) * C_X * (C_N / (C_N + 393.1))) if C_N <= 500 and C_X >= 10 else 0

        # Add additive disturbances to the dynamics
        disturbance = np.sin(self.time_step) * self.sigma_d + np.random.normal(0, self.sigma_d)
        
        dC_X += disturbance[0]
        dC_N += disturbance[1]
        dC_q += disturbance[2]

        # Update state with dynamics and disturbances
        self.state += np.array([dC_X, dC_N, dC_q])

        # Add measurement noise when observing the state
        noisy_state = self.state + np.random.normal(0, self.sigma_n)

        if self.time_step < self.max_time_steps - 1:
            reward = -np.dot(action - np.zeros(2), self.penalty_coefficients * (action - np.zeros(2)))
        else:
            reward = self.state[2]  # Final product concentration (C_q)

        self.time_step += 1
        done = self.time_step >= self.max_time_steps

        return noisy_state, reward, done, {}

    def render(self):
        pass
